# Data Pre-Processing

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [4]:
import numpy as np
import pandas as pd

In [88]:
# Training Data Paths
training_data = pd.read_csv("analysis_data_sources\CAL10K_with_Spotify_WithFeatures.tab", sep='\t')

In [89]:
X = training_data.drop(columns=['Spotify_ID', 'Song', 'Genre'], axis=1)
y = training_data["Genre"]
print(X.shape, y.shape)

(24731, 14) (24731,)


In [90]:
X.head(1)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0.498,0.249,9.0,-13.619,0.0,0.0355,0.755,0.000009,0.0927,0.193,110.636,256240.0,4.0


In [91]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Build a Deep Neural Network

In [114]:
# Create an empty sequential model
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=14))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=147, activation='softmax'))


In [115]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
deep_model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [118]:
# Use the training data to fit (train) the model
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=120,
    shuffle=True,
    verbose=2
)

Epoch 1/120
580/580 - 1s - loss: 0.5017 - accuracy: 0.8239
Epoch 2/120
580/580 - 1s - loss: 0.5076 - accuracy: 0.8205
Epoch 3/120
580/580 - 1s - loss: 0.5017 - accuracy: 0.8220
Epoch 4/120
580/580 - 1s - loss: 0.4944 - accuracy: 0.8273
Epoch 5/120
580/580 - 1s - loss: 0.4967 - accuracy: 0.8235
Epoch 6/120
580/580 - 1s - loss: 0.5007 - accuracy: 0.8222
Epoch 7/120
580/580 - 1s - loss: 0.4978 - accuracy: 0.8239
Epoch 8/120
580/580 - 1s - loss: 0.5008 - accuracy: 0.8261
Epoch 9/120
580/580 - 1s - loss: 0.4925 - accuracy: 0.8280
Epoch 10/120
580/580 - 1s - loss: 0.4951 - accuracy: 0.8274
Epoch 11/120
580/580 - 1s - loss: 0.4902 - accuracy: 0.8268
Epoch 12/120
580/580 - 1s - loss: 0.4873 - accuracy: 0.8288
Epoch 13/120
580/580 - 1s - loss: 0.4863 - accuracy: 0.8295
Epoch 14/120
580/580 - 1s - loss: 0.4919 - accuracy: 0.8276
Epoch 15/120
580/580 - 1s - loss: 0.4793 - accuracy: 0.8313
Epoch 16/120
580/580 - 1s - loss: 0.4895 - accuracy: 0.8295
Epoch 17/120
580/580 - 1s - loss: 0.4807 - accura

# Save the Trained Model

In [124]:
# Save the model
deep_model.save('genre_trained.h5')

# Evaluate the Model

In [131]:
# Load the model
from tensorflow.keras.models import load_model
deep_model = load_model("genre_trained.h5")

In [135]:
# Evaluate the model using the training data
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


194/194 - 0s - loss: 1.2132 - accuracy: 0.7469
Normal Neural Network - Loss: 1.213161587715149, Accuracy: 0.7468866109848022


In [153]:

encoded_predictions = deep_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)


In [154]:
print(f"Predicted: {list(prediction_labels)}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted: ['alternative', 'funk', 'country', 'soul', 'alternative']
Actual Labels: ['alternative', 'hip hop', 'country', 'outside lands music festival radio', 'alternative country']
